In [ ]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingClassifier, RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import time
start_time = time.time()

In [ ]:
X = np.load(r'E:\data2\2698NUM_TI_AB.npy')
y = np.load(r'E:\data2\label_012.npy')
y = np.array(y).ravel()
# 创建模拟数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42,stratify=y)
# 定义基模型
base_models = [
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('et', ExtraTreesClassifier(n_estimators=100, random_state=42)),
    ('ada', AdaBoostClassifier(n_estimators=100)),
    ('knn', KNeighborsClassifier()),
    ('mlp', MLPClassifier(max_iter=200)),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')),
    ('lgbm', LGBMClassifier()),
    ('catboost', CatBoostClassifier(verbose=0))
]

# 定义元模型
meta_model = LogisticRegression()

# 创建Stacking模型
stacking_clf = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5)

# 训练Stacking模型
stacking_clf.fit(X_train, y_train)
# 记录训练结束时间
end_time = time.time()

# 计算训练时间（秒）
training_time_seconds = end_time - start_time

# 转换训练时间为小时
training_time_hours = training_time_seconds / 3600

print(f"Model training took {training_time_hours:.3f} hours.")
# 评估模型性能
score = stacking_clf.score(X_test, y_test)
print(f"Stacking Model Accuracy: {score:.4f}")
def evaluate_classification_model(y_true, y_pred, dataset_name="Test"):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    conf_matrix = confusion_matrix(y_true, y_pred)
    
    print(f"{dataset_name} set:")
    print(f"  Accuracy: {accuracy}")
    print(f"  Precision: {precision}")
    print(f"  Recall: {recall}")
    print(f"  F1 Score: {f1}")
    print(f"  Confusion Matrix:\n{conf_matrix}\n")
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
# 对测试集进行预测
y_test_pred = stacking_clf.predict(X_test)
# 评估测试集
evaluate_classification_model(y_test, y_test_pred, "Test")